$$\textrm{Joaquin Peñuela Parra, Cristian Fernando Rodriguez Cruz}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

In [2]:
import os
import subprocess
from itertools import product
## IMPORTANT: Make sure that "Uniandes_Framework" is in .gitignore
framework_path = "Uniandes_Framework"

if os.path.exists(framework_path):
    # Pull updates if the framework is already cloned
    try:
        subprocess.run(["git", "-C", framework_path, "pull"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while pulling updates from the framework: {e}")
else:
    # Clone the framework if it is not already cloned
    try:
        subprocess.run(["git", "clone", "git@github.com:Phenomenology-group-uniandes/Uniandes_Framework.git"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while cloning the framework: {e}")

Already up to date.


In [3]:
import os
os.getcwd()

'/home/pheno/github/lq_zprime/03_delphes_preselection'

In [4]:
import os
import time

from Uniandes_Framework.delphes_reader import DelphesLoader 
from Uniandes_Framework.delphes_reader import classifier 
from Uniandes_Framework.delphes_reader import root_analysis 
from Uniandes_Framework.delphes_reader import Quiet 

from ROOT import TChain
from ROOT import TLorentzVector

import numpy as np
import pandas as pd
from multiprocessing import Pool

Welcome to JupyROOT 6.22/06


In [5]:
case = "woRHC"
# case = "wRHC"

limits = ['zp_upper_limit', 'zp_lower_limit']

# limit = 'zp_upper_limit'
# limit = 'zp_lower_limit'

In [6]:
from random import random
from time import sleep
def event_selection(x):
    #add random sleep to fix printing issues in multiprocessing pool
    sleep(2+random()*5)

    signal, m, gu, gzp,limit = x
    signal_key = f"{case}_{limit}_" + signal + f"_mu_{m}_gu{gu}_gzp_{gzp}".replace('.', '_')
    
    # csv_folder_path = f'/disco4/pheno_csv_files/Leptoquarks_Searches/{signal_key}/'
    csv_folder_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "csv_files", "lq_zp", signal, case, limit, f"mu_{m}_gu{gu}_gzp_{gzp}".replace('.', '_'))
    os.makedirs(csv_folder_path, exist_ok=True)

    print("-"*100, flush=True)
    print(f"Processing {signal_key} signal", flush=True)
    print("-"*100, flush=True)
    with Quiet():
    
        CUTS={"l_jet":     {"pt_min_cut":30., "eta_min_cut":-5.,  "eta_max_cut":+5. },
              "b_jet":     {"pt_min_cut":30., "eta_min_cut":-2.4, "eta_max_cut":+2.4},
              "tau_jet":   {"pt_min_cut":50., "eta_min_cut":-2.3, "eta_max_cut":+2.3},
              "other_jet": {"pt_min_cut":0.,  "eta_min_cut":-5.,  "eta_max_cut":+5.},
              "electron":  {"pt_min_cut":35.,  "eta_min_cut":-2.4, "eta_max_cut":+2.4},
              "muon":      {"pt_min_cut":30.,  "eta_min_cut":-2.4, "eta_max_cut":+2.4}}

        ST = []
        
        Delphes = DelphesLoader(signal_key,os.path.join(os.getcwd(), "SimulationsPaths.csv"))
        Paths = Delphes.Forest
        xs = Delphes.xs
        n_events = Delphes.get_nevents()
        print("Number of trees: ", len(Paths))
        print("Number of events: ", n_events)
        print("Cross section: ", xs)
        

        #We want a Cutflows file for each cathegory:        
        Cutflows = {'hadronic_non-resonant': {'XS': xs},
                    'hadronic_sLQ': {'XS': xs},
                    'hadronic_dLQ': {'XS': xs},
                    'semileptonic_non-resonant': {'XS': xs},
                    'semileptonic_sLQ': {'XS': xs},
                    'semileptonic_dLQ': {'XS': xs}}
        
        kinematic_variables = {}
        for key in Cutflows.keys(): kinematic_variables[key] = []
        i=0
        for path in Paths:
            i+=1
            tree = TChain("Delphes;1") 
            tree.Add(path) 
            print("Working on:\n", os.path.dirname(path))
            for event in tree:
                
                for key in Cutflows.keys(): Cutflows[key]['All'] = Cutflows[key].get('All',0) + 1
                    
                leptons = classifier.get_good_leptons(event, kin_cuts= CUTS)
                
                if(len(leptons) > 1): continue
                for key in Cutflows.keys(): Cutflows[key]['Maximum 1 light lepton'] = Cutflows[key].get('Maximum 1 light lepton',0) + 1
                           
                jets = classifier.get_good_jets(event, kin_cuts= CUTS)
                
                #Adding tau_jets like leptons:
                leptons = leptons + jets['tau_jet'] 
                
                if(len(leptons) != 2): continue 
                for key in Cutflows.keys(): Cutflows[key]['Exactly 2 leptons'] = Cutflows[key].get('Exactly 2 leptons',0) + 1
                
                #At this point, we have two hadronic taus or one hadronic tau and one lepton. Now, we can clasificate:
                
                #clasification:
                if(len(jets['tau_jet']) == 1): 
                    label1 = 'semileptonic'
                else:
                    label1 = 'hadronic'  
                
                    
                cut = f'{label1} selection'    
                for key in Cutflows.keys(): 
                    if label1 in key: Cutflows[key][cut] = Cutflows[key].get(cut, 0) + 1
                
                if(len(jets['b_jet']) == 0):
                    label2 = 'non-resonant'
                elif(len(jets['b_jet']) == 1):
                    label2 = 'sLQ'
                elif(len(jets['b_jet']) == 2):
                    label2 = 'dLQ'
                else: continue
                
                label = f'{label1}_{label2}'  
                
                cut = f'{label2} selection'    
                Cutflows[label][cut] = Cutflows[label].get(cut, 0) + 1                
                
                if(leptons[0].DeltaR(leptons[1]) < 0.3): continue #DeltaR > 0.3
                cut = 'DeltaR > 0.3'
                Cutflows[label][cut] = Cutflows[label].get(cut,0) + 1
                
                MET = classifier.get_met(event)
                MET.SetName('MET')
                particles = leptons + jets['b_jet'] + [MET]
                
                ST = 0
                for particle in particles: ST += particle.pt
                
                HT = 0
                for particle in jets['l_jet']: HT += particle.pt
                
                TLV = TLorentzVector(0,0,0,0)
                for particle in particles: TLV += particle.GetTLV()
                MT = TLV.Mt()
                
                Mass_invariant = (leptons[0].GetTLV() + leptons[1].GetTLV()).M()
                
                row = root_analysis.get_kinematics_row(particles)
                
                keylist = list(row.keys())
                for key in keylist: 
                    if ("Mass" in key): row.pop(key,None)
                
                row["sT(GeV)"] = ST
                row["hT(GeV)"] = HT
                row["mT(GeV)"] = MT
                row["light_jets_multiplicity"] = len(jets['l_jet'])
                name = leptons[0].Name + leptons[1].Name
                row[f"Mass_{{{name}}}(GeV)"] = Mass_invariant
                row[f"Q_{{{leptons[0].Name}}}Q_{{{leptons[1].Name}}}"] = leptons[0].Charge*leptons[1].Charge
                
                
                kinematic_variables[label].append(row)
                
        os.makedirs(csv_folder_path, exist_ok=True)
        for key in Cutflows.keys(): root_analysis.generate_csv(kinematic_variables[key], os.path.join(csv_folder_path, f'{signal_key}_{key}.csv'))
    print("-"*100)
    print("Ready: ", signal_key, " signal!")
    print("-"*100)
    return {signal_key: Cutflows}

In [7]:
start_time = time.time()
import pandas as pd

signals = [
    "total_tau_tau",
    ]

M_U= [1000, 1500, 2000, 2500, 3000, 3500]
G_U = [1.75]
G_ZP = [0.5, 1.5, 2.5, 3.5]

csv_sim_path = pd.read_csv(os.path.join("Uniandes_Framework", "SimulationsPaths.csv"))


df = pd.concat(
    [csv_sim_path]
    +
    [pd.DataFrame.from_dict(
    {
        "name": f"{case}_{limit}_" + signal + f"_mu_{m}_gu{gu}_gzp_{gzp}".replace('.', '_'), 
        "path": os.path.join(os.sep, 'disco2', 'SIMULATIONS', 'lq_zp', case, limit, signal,f'mu_{m}_gu{gu}_gzp_{gzp}'.replace('.', '_')),  
        'xs(pb)': pd.read_csv(os.path.join( os.path.dirname(os.getcwd()), "01_signal_production","xs_13TeV",case,limit, signal, f"XS_matrix_{gu}.csv"), index_col=0)[str(m)][gzp]
    }, 
    orient = "index").T for signal, m , gu, gzp, limit in product(signals,M_U,G_U, G_ZP,limits) ]
)

# save the new csv
csv_sim_path = os.path.join(os.getcwd(), "SimulationsPaths.csv")
df.to_csv(csv_sim_path, index = False)

# The main delay is the I/O from hard drive, 
# if you have a SSD, you can increase the number of workers, but be careful with the RAM
# I don't recommend use a SSD for Data because TBW (TeraBytes Written) is limited
n_workers = 4

with Pool(n_workers) as p:
    print("="*120)
    print("Starting with ", n_workers, " workers")
    print("="*120)
    # random sleep to avoid multiple printing problems
    mapping = list(p.map(event_selection, product(signals,M_U,G_U, G_ZP,limits )))
    print("="*120)
    print("Done!")
    print("Elapsed time in hours: ", (time.time() - start_time)/3600)
    print("="*120)

Starting with  4  workers
----------------------------------------------------------------------------------------------------
Processing woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_3_5 signal
----------------------------------------------------------------------------------------------------
woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_3_5 imported with 6 trees!
/disco2/SIMULATIONS/lq_zp/woRHC/zp_upper_limit/total_tau_tau/mu_1000_gu1_75_gzp_3_5
----------------------------------------------------------------------------------------------------
Processing woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_0_5 signal
----------------------------------------------------------------------------------------------------
woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_0_5 imported with 7 trees!
/disco2/SIMULATIONS/lq_zp/woRHC/zp_upper_limit/total_tau_tau/mu_1000_gu1_75_gzp_0_5
Number of trees:  6
Number of events:  150000
Cross section:  0.05684
Working on:
 /disc

In [8]:
labels = ['hadronic_non-resonant','hadronic_sLQ','hadronic_dLQ','semileptonic_non-resonant','semileptonic_sLQ','semileptonic_dLQ']

Dict_Cutflows = {}

for label in labels: Dict_Cutflows[label] = {}

for Cutflows_Signal_Directory in mapping:
    
    signal = list(Cutflows_Signal_Directory.keys())[0]
    
    for label in labels: Dict_Cutflows[label][signal] = Cutflows_Signal_Directory[signal][label]

In [9]:
Dict_DataFrames_Cutflows = {}

for key in Dict_Cutflows: 
    Dict_DataFrames_Cutflows[key] = pd.DataFrame.from_dict(Dict_Cutflows[key])
    cutflows_folder_path = os.path.join(os.getcwd(), 'Cutflows')
    cutflow_path = os.path.join(cutflows_folder_path, f'cutflow_{key}.csv')
    os.makedirs(cutflows_folder_path, exist_ok=True)
    Dict_DataFrames_Cutflows[key].to_csv(cutflow_path)

In [10]:
Dict_DataFrames_Cutflows.keys()

dict_keys(['hadronic_non-resonant', 'hadronic_sLQ', 'hadronic_dLQ', 'semileptonic_non-resonant', 'semileptonic_sLQ', 'semileptonic_dLQ'])

In [11]:
Dict_DataFrames_Cutflows["hadronic_non-resonant"]

,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_0_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_0_5,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_1_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_1_5,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_2_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_2_5,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_3_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_3_5,woRHC_zp_upper_limit_total_tau_tau_mu_1500_gu1_75_gzp_0_5,woRHC_zp_lower_limit_total_tau_tau_mu_1500_gu1_75_gzp_0_5,...,woRHC_zp_upper_limit_total_tau_tau_mu_3000_gu1_75_gzp_3_5,woRHC_zp_lower_limit_total_tau_tau_mu_3000_gu1_75_gzp_3_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_0_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_0_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_1_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_1_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_2_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_2_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_3_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_3_5
XS,0.043,0.06001,0.04233,0.2005,0.04627,0.491,0.05684,0.8385,0.009017,0.01117,...,9.015e-05,0.003053,0.0003256,0.0003202,0.0002577,0.000252,0.0001473,0.0003809,4.393e-05,0.001221
All,175000,150000,150000,150000,150000,150000,150000,150000,175000,150000,...,150000,150000,150000,150000,175000,150000,150000,150000,150000,150000
Maximum 1 light lepton,169008,144015,144010,142241,142150,142239,140910,142363,168745,143785,...,143558,140889,144407,144172,168460,142001,144463,139661,144250,140922
Exactly 2 leptons,35300,34730,34826,42104,40542,42388,45643,41921,36945,35510,...,35475,45229,32589,33604,38000,40236,31483,48510,32961,43985
hadronic selection,16326,15565,15525,18462,17562,18623,19116,18356,16928,15704,...,15509,19112,14836,15085,16960,17171,14334,19825,14723,18636
non-resonant selection,11578,10960,10918,12708,12157,12794,12948,12690,12105,11007,...,11030,12931,10542,10607,12110,11924,10209,13381,10360,12798
DeltaR > 0.3,11578,10960,10918,12708,12157,12794,12948,12690,12105,11007,...,11030,12931,10542,10607,12110,11924,10209,13381,10360,12798


In [12]:
Dict_DataFrames_Cutflows["hadronic_sLQ"]

,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_0_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_0_5,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_1_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_1_5,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_2_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_2_5,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_3_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_3_5,woRHC_zp_upper_limit_total_tau_tau_mu_1500_gu1_75_gzp_0_5,woRHC_zp_lower_limit_total_tau_tau_mu_1500_gu1_75_gzp_0_5,...,woRHC_zp_upper_limit_total_tau_tau_mu_3000_gu1_75_gzp_3_5,woRHC_zp_lower_limit_total_tau_tau_mu_3000_gu1_75_gzp_3_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_0_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_0_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_1_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_1_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_2_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_2_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_3_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_3_5
XS,0.043,0.06001,0.04233,0.2005,0.04627,0.491,0.05684,0.8385,0.009017,0.01117,...,9.015e-05,0.003053,0.0003256,0.0003202,0.0002577,0.000252,0.0001473,0.0003809,4.393e-05,0.001221
All,175000,150000,150000,150000,150000,150000,150000,150000,175000,150000,...,150000,150000,150000,150000,175000,150000,150000,150000,150000,150000
Maximum 1 light lepton,169008,144015,144010,142241,142150,142239,140910,142363,168745,143785,...,143558,140889,144407,144172,168460,142001,144463,139661,144250,140922
Exactly 2 leptons,35300,34730,34826,42104,40542,42388,45643,41921,36945,35510,...,35475,45229,32589,33604,38000,40236,31483,48510,32961,43985
hadronic selection,16326,15565,15525,18462,17562,18623,19116,18356,16928,15704,...,15509,19112,14836,15085,16960,17171,14334,19825,14723,18636
sLQ selection,4265,4084,4122,5126,4776,5202,5402,5053,4348,4191,...,3984,5430,3848,4006,4342,4645,3692,5652,3900,5163
DeltaR > 0.3,4265,4084,4122,5126,4776,5202,5402,5053,4348,4191,...,3984,5430,3848,4006,4342,4645,3692,5652,3900,5163


In [31]:
Dict_DataFrames_Cutflows['hadronic_dLQ']

,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_0_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_0_5,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_1_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_1_5,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_2_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_2_5,woRHC_zp_upper_limit_total_tau_tau_mu_1000_gu1_75_gzp_3_5,woRHC_zp_lower_limit_total_tau_tau_mu_1000_gu1_75_gzp_3_5,woRHC_zp_upper_limit_total_tau_tau_mu_1500_gu1_75_gzp_0_5,woRHC_zp_lower_limit_total_tau_tau_mu_1500_gu1_75_gzp_0_5,...,woRHC_zp_upper_limit_total_tau_tau_mu_3000_gu1_75_gzp_3_5,woRHC_zp_lower_limit_total_tau_tau_mu_3000_gu1_75_gzp_3_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_0_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_0_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_1_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_1_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_2_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_2_5,woRHC_zp_upper_limit_total_tau_tau_mu_3500_gu1_75_gzp_3_5,woRHC_zp_lower_limit_total_tau_tau_mu_3500_gu1_75_gzp_3_5
XS,0.043,0.06001,0.04233,0.2005,0.04627,0.491,0.05684,0.8385,0.009017,0.01117,...,9.015e-05,0.003053,0.0003256,0.0003202,0.0002577,0.000252,0.0001473,0.0003809,4.393e-05,0.001221
All,175000,150000,150000,150000,150000,150000,150000,150000,175000,150000,...,150000,150000,150000,150000,175000,150000,150000,150000,150000,150000
Maximum 1 light lepton,169008,144015,144010,142241,142150,142239,140910,142363,168745,143785,...,143558,140889,144407,144172,168460,142001,144463,139661,144250,140922
Exactly 2 leptons,35300,34730,34826,42104,40542,42388,45643,41921,36945,35510,...,35475,45229,32589,33604,38000,40236,31483,48510,32961,43985
hadronic selection,16326,15565,15525,18462,17562,18623,19116,18356,16928,15704,...,15509,19112,14836,15085,16960,17171,14334,19825,14723,18636
dLQ selection,458,499,461,605,596,594,719,593,450,482,...,469,714,427,450,482,576,417,756,445,631
DeltaR > 0.3,458,499,461,605,596,594,719,593,450,482,...,469,714,427,450,482,576,417,756,445,631


In [37]:
import shutil
eff_dict = {key: {} for key in Dict_DataFrames_Cutflows.keys()}

efficiencies_folder_path = os.path.join(os.getcwd(), "Efficiencies")
#clean the folder
shutil.rmtree(efficiencies_folder_path, ignore_errors=True)

for limit, signal, gu, channel in product(limits, signals,G_U, Dict_DataFrames_Cutflows.keys()):
    eff_matrix = pd.DataFrame(index = G_ZP, columns = M_U)
    for m, gzp in product(M_U, G_ZP):
        column_name = "_".join( [case, limit ,  signal , f"mu_{m}_gu{gu}_gzp_{gzp}".replace('.', '_')])
        eff_matrix[m][gzp] = Dict_DataFrames_Cutflows[channel][column_name]["DeltaR > 0.3"]/Dict_DataFrames_Cutflows[channel][column_name]["All"]
    eff_dict[channel][str(gu)] = eff_matrix    
    eff_out_path = os.path.join(efficiencies_folder_path, case, limit, signal,channel)
    os.makedirs(eff_out_path, exist_ok=True)
    eff_matrix.to_csv(os.path.join(eff_out_path, f"eff_matrix_{gu}.csv"))

In [32]:
os.getcwd()

'/home/pheno/github/lq_zprime/03_delphes_preselection'